In [1]:
from tensorflow.keras.models import load_model
import pickle
import pandas as pd

In [2]:
## Load the ANN trained model

model = load_model("model.h5") 

## standard scaler 

with open("stdsclr.pkl", "rb") as file:
    stdsclr = pickle.load(file)

## load one hot encoded file

with open("ohe_geo.pkl","rb") as file:
    ohe_geo = pickle.load(file)

## load label encoded file for gender

with open("label_encoder_gender.pkl","rb") as file:
   label_encoder_gender = pickle.load(file)



In [3]:
# Sample input data

input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}

In [7]:
input_df = pd.DataFrame([input_data])

In [8]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [18]:
# Lets OHE the Geography column with the pickled ohe_geo 

geo_encoded = ohe_geo.transform([input_df["Geography"]]).toarray()

j:\data_science_projects\deep learning\ANN\annclassification\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [19]:
geo_encoded_df = pd.DataFrame(geo_encoded,columns=ohe_geo.get_feature_names_out(["Geography"]))

In [20]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [21]:
# Lets label encode the gender column with the pickeled lebel encoder gender

input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])

In [22]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [29]:
# Lets concatinate ohe geo df with input_df

input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [30]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [31]:
# Lets apply standard scaler to input_df

input_scaled = stdsclr.transform(input_df)



In [32]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [33]:
# Prediction time

prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 286ms/step


array([[0.03564391]], dtype=float32)

In [34]:
prediction_proba = prediction[0][0]
prediction_proba

0.035643913

In [35]:
if prediction_proba > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
